## Configuração do ambiente

In [1]:
from sshtunnel import open_tunnel
import pandas as pd
import yaml
import pymongo

In [2]:
# Obtendo as credenciais de acesso ao banco de dados
with open('config.yaml', 'r') as f:
    configs = yaml.safe_load(f)
lamcad_configs = configs['lamcad']
mongodb_configs = configs['mongodb_lamcad']

In [3]:
# Conectando ao LaMCAD
server = open_tunnel(
    (lamcad_configs['server_ip'], lamcad_configs['server_port']),
    ssh_username=lamcad_configs['ssh_username'],
    ssh_password=lamcad_configs['ssh_password'],
    local_bind_address=(lamcad_configs['local_bind_ip'], lamcad_configs['local_bind_port']),
    remote_bind_address=(lamcad_configs['remote_bind_ip'], lamcad_configs['remote_bind_port'])
)
server.start()

# Conectando ao banco de dados
client = pymongo.MongoClient(mongodb_configs['uri'])
database = client[mongodb_configs['database']]

print(f"Conexão com o LaMCAD criada no seguinte IP local e porta: {server.local_bind_address}")
print("Não se esqueça de fechar a conexão com: server.close()!")

Conexão com o LaMCAD criada no seguinte IP local e porta: ('127.0.0.1', 27018)
Não se esqueça de fechar a conexão com: server.close()!


In [4]:
# Coleção de notícias aceitas e coleção de notícias não aceitas
an_collection = database[mongodb_configs['accepted_news_collection']]
un_collection = database[mongodb_configs['unaccepted_news_collection']]

## Exemplos de acessos ao banco

In [5]:
# Quantidade de notícias assinaladas manualmente como relevantes
query = {'manual_relevance_class': 1}
count = an_collection.count_documents(query)
print(count)

294


In [6]:
# Quantidade de notícias assinaladas manualmente como não relevantes
query = {'manual_relevance_class': 0}
count = an_collection.count_documents(query)
print(count)

4962


In [7]:
# Obtendo todas as notícias assinaladas manualmente como relevantes
# apenas os valores de "_id", "url", "newspaper", title" e "article" de cada notícia
pipeline = [
    {'$match': {'manual_relevance_class': 1}},
    {'$project': {
        '_id': 1,
        'newspaper': 1,
        'url': 1,
        'title': 1,
        'article': 1
    }}
]

relevant_news = list(an_collection.aggregate(pipeline))

print('Quantidade de resultados:', len(relevant_news))
print('Título do primeiro resultado:', relevant_news[0]['title'])

Quantidade de resultados: 294
Título do primeiro resultado: Mais de 400 suspeitos foram presos durante operação policial em MT


In [37]:
# Obtendo uma lista de N notícias aleatórias marcadas manualmente como não relevantes
# apenas os valores de "_id", "url", "newspaper", title" e "article" de cada notícia
N = 294
pipeline = [
    {'$match': {'manual_relevance_class': 0}},
    {"$sample": {"size": N}},
    {'$project': {
        '_id': 1,
        'newspaper': 1,
        'url': 1,
        'title': 1,
        'article': 1
    }}
]
non_relevant_news = list(an_collection.aggregate(pipeline))

print('Quantidade de resultados:', len(non_relevant_news))
print('Título do primeiro resultado:', non_relevant_news[0]['title'])

Quantidade de resultados: 294
Título do primeiro resultado: Traficante FB chega ao Rio, diz polícia


## Salvamento de dados do banco

In [8]:
# Salvando as notícias relevantes obtidas anteriormente em uma planilha
pd.DataFrame(relevant_news).to_excel('relevant_news.xlsx')

In [39]:
# Salvando as notícias não relevantes obtidas anteriormente em uma planilha
pd.DataFrame(relevant_news).to_excel('non_relevant_news.xlsx')

In [40]:
server.stop()

2024-06-02 09:11:37,825| ERROR   | Socket exception: An existing connection was forcibly closed by the remote host (10054)
2024-06-02 09:11:37,925| ERROR   | Socket exception: An existing connection was forcibly closed by the remote host (10054)
2024-06-02 09:11:37,978| ERROR   | Could not establish connection from local ('127.0.0.1', 27018) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel error: [WinError 10054] An existing connection was forcibly closed by the remote host
